In [ ]:
# imports
import os
import re
import json

import pandas as pd
from Bio.Seq import Seq

from eagle.lib.seqs import SeqsDict

In [ ]:
# constants
WORK_DIR = "bacteria"

essential_nucl_fasta = "bacteria/deg_essential/degseq-p.dat"
essential_annot = "bacteria/deg_essential/degannotation-p.dat"

nonessential_nucl_fasta = "bacteria/deg_nonessential/degseq-np.dat"
nonessential_annot = "bacteria/deg_nonessential/degannotation-np.dat"

chr_id_path = "bacteria/chr_id.txt"
summary_path = "bacteria/summary_table.txt"

In [ ]:
# lib
GTF_COLS = ["seqid", "source", "type", "start", "end", "score", "strand", "frame", "attribute"]

def get_gtfs(chr_dict, deg_annot_path, deg_seqs_fasta, deg_essential):
    annot_df = pd.read_csv(deg_annot_path, sep="\t")
    seqs_dict = SeqsDict.load_from_file(deg_seqs_fasta, low_memory=True)

    gtfs_df = pd.DataFrame(list(annot_df.apply(prepare_deg_annot_line, axis=1, args=(chr_dict, seqs_dict, deg_essential))))
    summary_list = list()
    for deg_org in gtfs_df.groupby("org_DEG_ID"):
        summary_list.append(deg_org[1].iloc[0][["org_DEG_ID", "chr_id", "org_name"]].to_dict())
        gtf_df = deg_org[1][pd.notna(deg_org[1]["seqid"])][GTF_COLS]
        
        if deg_essential:
            gtf_path = os.path.join(WORK_DIR, deg_org[0]+"_essential.gtf")
            summary_list[-1].update({"essential_genes": gtf_df.shape[0], "essential_genes_gtf": gtf_path})
        else:
            gtf_path = os.path.join(WORK_DIR, deg_org[0]+"_nonessential.gtf")
            summary_list[-1].update({"nonessential_genes": gtf_df.shape[0], "nonessential_genes_gtf": gtf_path})
        if not gtf_df.empty:
            gtf_df.to_csv(gtf_path.replace("DNEG", "DEG"), sep="\t", index=False, quotechar="'")
        gtf_path = None
        
        print(summary_list[-1])
    return pd.DataFrame(summary_list)


def prepare_deg_annot_line(row, chr_dict, deg_seqs_dict, deg_essential):
    result_dict = {
        "org_DEG_ID": row["#DEG_ORG"],
        "chr_id": row["#Refseq"],
        "org_name": row["#Organism"],
        # gtf fields
        "seqid": None,
        "source": "DEG",
        "type": "gene",
        "start": int(),
        "end": int(),
        "score": "-",
        "strand": None,
        "frame": ".",
        "attribute": json.dumps({"gene_name": row["#Gene_Name"]}),
    }
    if row["#Refseq"] not in chr_dict:
        return result_dict    
    ori = 1
    match = search_in_chr(deg_seqs_dict[row["#DEG_AC"]], chr_dict[row["#Refseq"]])
    if match is None:
        if deg_essential:
            match = search_in_chr(str(Seq(deg_seqs_dict[row["#DEG_AC"]]).reverse_complement()), chr_dict[row["#Refseq"]])
        else:
            match = search_in_chr(deg_seqs_dict[row["#DEG_AC"]][::-1], chr_dict[row["#Refseq"]])
        ori = -1
    if match is not None:
        result_dict["seqid"] = match[0]
        result_dict["start"] = match[1] + 1
        result_dict["end"] = match[2]
        if ori > 0:
            result_dict["strand"] = "+"
        else:
            result_dict["strand"] = "-"
    else:
        print("WARNING: no match found for '%s'" % row["#DEG_AC"])
    return result_dict


def search_in_chr(seq, chr_seq_dict):
    for seq_name in chr_seq_dict:
        match = re.search(seq.lower(), chr_seq_dict[seq_name].lower())
        if match is not None:
            return seq_name.split()[0], match.start(), match.end()

        
def detect_gtf_intersect(row, inter_gtf_df):
    max_inter = 0.0
    query = 'seqid == "%s" & end >= %s & start <= %s' % (row["seqid"], row["start"], row["end"])
    for i, row_ in inter_gtf_df.query(query).iterrows():
        cur_inter = float(min(row["end"], row_["end"]) - max(row["start"], row_["start"]) + 1) / float(max(row["end"], row_["end"]) - min(row["start"], row_["start"]) + 1)
        if cur_inter > max_inter:
            max_inter = cur_inter
    return max_inter

In [ ]:
# main
chr_dict = dict()
chr_id_df = pd.read_csv(chr_id_path, sep="\t")
for i, chr_id in enumerate(chr_id_df["chr_id"]):
    chr_dict[chr_id] = SeqsDict.load_from_file(chr_id_df.iloc[i]["fna_path"], low_memory=True)

essential_summary_df = get_gtfs(chr_dict=chr_dict, deg_annot_path=essential_annot, deg_seqs_fasta=essential_nucl_fasta, deg_essential=True)
print("INFO: got essential summary")
nonessential_summary_df = get_gtfs(chr_dict=chr_dict, deg_annot_path=nonessential_annot, deg_seqs_fasta=nonessential_nucl_fasta, deg_essential=False)
nonessential_summary_df["org_DEG_ID"] = nonessential_summary_df["org_DEG_ID"].apply(lambda org_deg_id: org_deg_id.replace("DNEG", "DEG"))

summary_df = essential_summary_df.merge(nonessential_summary_df[["org_DEG_ID", "nonessential_genes", "nonessential_genes_gtf"]], on="org_DEG_ID")
summary_df_columns = ["org_DEG_ID", "chr_id", "org_name", "essential_genes", "essential_genes_gtf", "nonessential_genes", "nonessential_genes_gtf"]
summary_df = summary_df[summary_df_columns].merge(chr_id_df, how='left', on="chr_id")
summary_df[summary_df["essential_genes"] > 0].to_csv(os.path.join(WORK_DIR, "summary_table.txt"), sep="\t", index=False)

!rm ./.*.dat

In [ ]:
# nonessential gtf from ncbi feature_table.txt
feature_table_path = "bacteria/NC_000913_feature_table.txt"
deg_ess_gtf_path = "bacteria/DEG1018_essential.gtf"
out_gtf_path = "bacteria/DEG1018_nonessential_ncbi.gtf"

deg_ess_gtf_df = pd.read_csv(deg_ess_gtf_path, sep="\t")
ft_df = pd.read_csv(feature_table_path, sep="\t")
ncbi_gtf_df = ft_df[ft_df["# feature"] == "gene"][ft_df["class"] == "protein_coding"][["genomic_accession", "start", "end", "strand", "symbol"]].reset_index(drop=True)
ncbi_gtf_df["attribute"] = ncbi_gtf_df["symbol"].apply(lambda gn: json.dumps({"gene_name": gn}))
del ncbi_gtf_df["symbol"]
ncbi_gtf_df.rename(index=str, columns={"genomic_accession": "seqid"}, inplace=True)
ncbi_gtf_df["source"] = "NCBI"
ncbi_gtf_df["type"] = "gene"
ncbi_gtf_df["score"] = "-"
ncbi_gtf_df["frame"] = "."
ncbi_gtf_df = ncbi_gtf_df[GTF_COLS]
deg_ess_intersect = ncbi_gtf_df.apply(detect_gtf_intersect, axis=1, args=(deg_ess_gtf_df,))
ncbi_gtf_df["is_essential"] = deg_ess_intersect[deg_ess_intersect < 0.5]
ncbi_gtf_df.dropna()[GTF_COLS].to_csv(out_gtf_path, sep="\t", index=False)